In [ ]:
#This repo content all the dataset, the record and the config that were used in training a TensorFlow pedestrian detector model.
!git clone https://github.com/Tourenathan-G5organisation/Pedestrian-Detection

Cloning into 'Pedestrian-Detection'...
remote: Enumerating objects: 1935, done.
remote: Total 1935 (delta 0), reused 0 (delta 0), pack-reused 1935
Receiving objects: 100% (1935/1935), 151.00 MiB | 41.57 MiB/s, done.
Resolving deltas: 100% (931/931), done.


Note: To create tfrecord files from these .csv files generated above, we need to update the generate_tfrecord.py 

In [ ]:
#create the tfrecords files by running the command below from model/research/object_detection directory
python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=images/train.record
python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=images/test.record

In [ ]:
#configure how training will work before move forward. 
#Create a folder called training in the model/research/object_detection directory. 
#This folder will contain our training config files necessary for this training.
%cd Pedestrian-Detection/
%mkdir -p training

/content/Pedestrian-Detection


In [ ]:
%%writefile -a labelmap.pbtxt
item {
  id: 1
  name: 'person'
}

Writing labelmap.pbtxt


There are two ways to customize a pretrained model:

**Feature Extraction**: Use the representations learned by a previous network to extract meaningful features from new samples. You simply add a new classifier, which will be trained from scratch, on top of the pretrained model so that you can repurpose the feature maps learned previously for the dataset.

You do not need to (re)train the entire model. The base convolutional network already contains features that are generically useful for classifying pictures. However, the final, classification part of the pretrained model is specific to the original classification task, and subsequently specific to the set of classes on which the model was trained.

**Fine-Tuning**: Unfreeze a few of the top layers of a frozen model base and jointly train both the newly-added classifier layers and the last layers of the base model. This allows us to "fine-tune" the higher-order feature representations in the base model in order to make them more relevant for the specific task.

Sources: [Tensorflow docs](https://www.tensorflow.org/tutorials/images/transfer_learning)

For this task we’ll use **Single Shot Detector(SSD)** with **MobileNet** (model optimized for inference on mobile) pretrained on the **COCO dataset** called *ssd_mobilenet_v2_quantized_coco*. First, We will download and extract the latest checkpoint that’s been pre-trained on the COCO dataset. Run the command below from object_detection directory

In [ ]:
%mkdir -p Pedestrian-Detection/pretrained/
%cd Pedestrian-Detection/pretrained/
!curl -O http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz 
!tar xzf ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
%rm ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
%cd ../..
%ls Pedestrian-Detection/pretrained

[Errno 2] No such file or directory: 'Pedestrian-Detection/pretrained /'
/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  138M  100  138M    0     0   193M      0 --:--:-- --:--:-- --:--:--  193M
tar (child): pretrained/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
rm: cannot remove 'pretrained/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz': No such file or directory
/
ls: cannot access 'Pedestrian-Detection/pretrained': No such file or directory


**Training configuration file**: we will use as config file *ssd_mobilenet_v2_quantized_300x300_coco.config* which can be found at sample/config folder in the object_detection folder.

update the file as follows:

* Line 9: change the number of classes:

```
num_classes: 1
```

* Line 156: Update fine_tune_checkpoint to the path of model.ckpt

```
fine_tune_checkpoint: "/content/Pedestrian-Detection/pretrained/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt"
```

* Line 175: Update the path of the train.record file

```
input_path: "/content/Pedestrian-Detection/images/train*.record"
```

* Line 177 & 192: update the path to the path of labelmap.pbtxt

```
label_map_path: "/content/Pedestrian-Detection/training/labelmap.pbtxt"
```

* Line 190: Update the path of the test.record file

```
input_path: "/content/Pedestrian-Detection/images/test*.record"
```

* Add the line below to the eval_config if you want to use the coco evaluation metrics

```
metrics_set: "coco_detection_metrics"
```

* Line 181: Update the number of examples to the number of images in your test directory.

```
num_examples: 235
```

**Note: use the `config_pipeline.py` script to update automatically**

# Training model

In [ ]:
#To use  use COCO evaluation metrics to evaluate the accuracy of our model during training. So we installed COCO APIs as follows:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
%cp -r pycocotools <path_to_tensorflow>/models/research/
%cd ../..

In [49]:
!pip3 install tf_slim
!apt-get install -y protobuf-compiler python-pil python-lxml python-tk
!pip3 install --user cython
!pip3 install --user contextlib2
!pip install tf-models-official

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 21 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [ ]:
#To train the model:
!git clone https://github.com/tensorflow/models.git
%cp models/research/object_detection/legacy/train.py models/research/object_detection/

In [50]:
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!python setup.py build
!python setup.py install

/content/models/research
running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/export_tflite_graph_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/model_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/exporter_tf1_test.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/model_main_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/ex

In [62]:
!python object_detection/builders/model_builder_test.py

2020-10-28 18:56:10.108147: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [70]:
%cd /content/Pedestrian-Detection

/content/Pedestrian-Detection


In [72]:
%env PYTHONPATH=$PYTHONPATH:'/content/models/research':'/content/models/research/slim'
%env PYTHONPATH=$PYTHONPATH:'/content/models'
#if it does not work add the following to train.py
#sys.path.insert(1, '/content/models/research/')
#sys.path.insert(1, '/content/models/research/slim')
#sys.path.insert(1, '/content/models/')
!python /content/models/research/object_detection/train.py --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v2_quantized_300x300_coco.config

env: PYTHONPATH=$PYTHONPATH:'/content/models/research':'/content/models/research/slim'
env: PYTHONPATH=$PYTHONPATH:'/content/models'
2020-10-28 19:02:30.103729: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
Use object_detection/model_main.py.
W1028 19:02:32.366685 140166026143616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:251: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.
Traceback (most recent call last):
  File "/content/models/research/object_detection/train.py", line 188, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 300, in run
    _run_main(m